In [1]:
import uproot
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, schemas
from coffea import processor, hist

## Define a Processor instance

The processor implementation given below has been taken from [here](https://github.com/CoffeaTeam/coffea/blob/master/binder/nanoevents.ipynb).

In [2]:
class MyZPeak(processor.ProcessorABC):
    def __init__(self):
        self._histo = hist.Hist(
            "Events",
            hist.Cat("dataset", "Dataset"),
            hist.Bin("mass", "Z mass", 60, 60, 120),
        )
    
    @property
    def accumulator(self):
        return self._histo
    
    # we will receive a NanoEvents instead of a coffea DataFrame
    def process(self, events):
        out = self.accumulator.identity()
        mmevents = events[
            (ak.num(events.Muon) == 2)
            & (ak.sum(events.Muon.charge, axis=1) == 0)
        ]
        zmm = mmevents.Muon[:, 0] + mmevents.Muon[:, 1]
        out.fill(
            dataset=events.metadata["dataset"],
            mass=zmm.mass,
        )
        return out
    
    def postprocess(self, accumulator):
        return accumulator

## Write some NanoEvents Parquet files to CephFS

Here we populate the CephFS mounted directory with the parquet files created in the previous step. In this version, we need to make sure that the individual file sizes is under 4MB which is the default object size of Ceph to ensure one-to-one mapping of files to objects, which is a requirement in the multiple-file design that we have now.

In [3]:
import os, shutil

if not os.path.isfile("nano_dy.parquet"):
    ak.to_parquet(
        uproot.lazy("nano_dy.root:Events"),
        "nano_dy.parquet",
        list_to32=True,
        use_dictionary=False,
        compression="GZIP",
        compression_level=1,
    )

if not os.path.isfile("nano_dimuon.parquet"):
    ak.to_parquet(
        uproot.lazy("nano_dimuon.root:Events"),
        "nano_dimuon.parquet",
        list_to32=True,
        use_dictionary=False,
        compression="GZIP",
        compression_level=1,
    )
    
if not os.path.isdir("/mnt/cephfs/nanoevents/ZJets"):
    os.makedirs("/mnt/cephfs/nanoevents/ZJets")
    for i in range(2):
        shutil.copyfile('nano_dy.parquet', '/mnt/cephfs/nanoevents/ZJets/nano_dy.{}.parquet'.format(i))
        
    
if not os.path.isdir("/mnt/cephfs/nanoevents/Data"):
    os.makedirs("/mnt/cephfs/nanoevents/Data")
    for i in range(2):
        shutil.copyfile('nano_dimuon.parquet', '/mnt/cephfs/nanoevents/Data/nano_dimuon.{}.parquet'.format(i))

## Running a job in parallel using Dask

The `LocalCluster()` used below creates a process pool with worker count equal to the number of cores available to the Notebook where each worker is single-threaded. The `LocalCluster` can be replaced by other cluster resource managers provided by Dask Distributed like `KuberneresCluster`, `YarnCluster`, etc. Here, we create a `LocalCluster` and get a client handle to it.

In [4]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:38983")
client

<Client: 'tcp://127.0.0.1:38983' processes=1 threads=4, memory=15.70 GiB>

We have added a new function called `run_parquet_job` to the executor API in coffea to run jobs on Parquet files using the Arrow Dataset API under the hood. 
This API takes an optional `ceph_config_path` parameter, which is basically the path to the configuration file of the Ceph cluster and instructs this function to read from RADOS using the `SkyhookFileFormat` (which allows pushdown) instead of the out of the box `ParquetFormat` API . This API also allows just passing a single directory path and the Datasets API does the dataset discovery task by itself. The calls to the Dataset API are launced in parallel and there will one Dataset API call per file.

In [5]:
%%time

run = processor.Runner(
    executor=processor.DaskExecutor(client=client),
    use_skyhook=True,
    #Savemetrics is not supported: UnboundLocalError: local variable 'materialized' referenced before assignment
    #savemetrics=True,
    skyhook_options = {"ceph_config_path": "/opt/ceph/ceph.conf", "ceph_data_pool": "cephfs_data0"},
    format="parquet",
    schema=schemas.NanoAODSchema,
)

hists = run(
    {
        "ZJets": "/mnt/cephfs/nanoevents/ZJets",
        "Data": "/mnt/cephfs/nanoevents/Data",
    },
    "Events",
    processor_instance=MyZPeak(),
)

RuntimeError: Work item WorkItem(dataset='ZJets', filename='/opt/ceph/ceph.conf:cephfs_data0:/mnt/cephfs/nanoevents/ZJets/nano_dy.1.parquet', treename='Events', entrystart=0, entrystop=0, fileuuid='', usermeta=None) caused a KilledWorker exception (likely a segfault or out-of-memory issue)

## Running iteratively using the `FuturesExecutor`

Run the same job again, but now iteratively. The calls to the Dataset API will now be sequential.

In [5]:
%%time

run = processor.Runner(
    executor=processor.FuturesExecutor(compression=None),
    use_skyhook=True,
    # Savemetrics is not supported: UnboundLocalError: local variable 'materialized' referenced before assignment
    #savemetrics=True,
    skyhook_options = {"ceph_config_path": "/opt/ceph/ceph.conf", "ceph_data_pool": "cephfs_data0"},
    format="parquet",
    schema=schemas.NanoAODSchema,
)

hists = run(
    {
        "ZJets": "/mnt/cephfs/nanoevents/ZJets",
        "Data": "/mnt/cephfs/nanoevents/Data",
    },
    "Events",
    processor_instance=MyZPeak(),
)

Processing:   0%|          | 0/2 [00:00<?, ?chunk/s]

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

## Running iteratively without Skyhook `FuturesExecutor`

Run the same job again, but now iteratively without Skyhook. The calls to the Dataset API will now be sequential.

In [ ]:
%%time

run = processor.Runner(
    executor=processor.IterativeExecutor(compression=None),
    use_skyhook=False,
    # Savemetrics is not supported: UnboundLocalError: local variable 'materialized' referenced before assignment
    #savemetrics=True,
    skyhook_options = {"ceph_config_path": "/opt/ceph/ceph.conf", "ceph_data_pool": "cephfs_data0"},
    format="parquet",
    schema=schemas.NanoAODSchema,
)

hists = run(
    {
        "ZJets": "/mnt/cephfs/nanoevents/ZJets",
        "Data": "/mnt/cephfs/nanoevents/Data",
    },
    "Events",
    processor_instance=MyZPeak(),
)

As expected, much slower than running using Dask.

## Running `DaskExecutor` without Skyhook

Run the same job again, but with `DaskExecutor` abd without Skyhook:

In [ ]:
%%time

run = processor.Runner(
    executor=processor.DaskExecutor(client=client),
    use_skyhook=False,
    # Savemetrics is not supported: UnboundLocalError: local variable 'materialized' referenced before assignment
    #savemetrics=True,
    skyhook_options = {"ceph_config_path": "/opt/ceph/ceph.conf", "ceph_data_pool": "cephfs_data0"},
    format="parquet",
    schema=schemas.NanoAODSchema,
)

hists = run(
    {
        "ZJets": "/mnt/cephfs/nanoevents/ZJets",
        "Data": "/mnt/cephfs/nanoevents/Data",
    },
    "Events",
    processor_instance=MyZPeak(),
)

## Plotting the results



In [ ]:
%matplotlib inline

hist.plot1d(hists)